In [84]:
# GPU 사용 여부 확인
import torch
torch.cuda.is_available()  # True면 GPU 사용 가능

# 필요 라이브러리 설치 (보통 Colab에 기본 설치되어 있음)
!pip install torch torchvision pillow

In [85]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

train_dir = '/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = '/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'



Mounted at /content/drive


In [86]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 이미지 전처리 (학습용과 검증용)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 데이터셋 로드
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from torchvision.models import resnet18, ResNet18_Weights

# ---------------------------
# 1. 데이터셋 준비
# ---------------------------
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 이미지 크기 줄여서 속도 향상
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train', transform=transform)
valid_dataset = datasets.ImageFolder('/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid', transform=transform)

# 테스트용: 데이터 일부만 사용 (속도 개선)
train_loader = DataLoader(Subset(train_dataset, range(min(500, len(train_dataset)))), batch_size=16, shuffle=True)
valid_loader = DataLoader(Subset(valid_dataset, range(min(100, len(valid_dataset)))), batch_size=16)



In [ ]:
# 2. 모델 준비
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

Using device: cuda


In [ ]:
# 3. 손실함수와 옵티마이저
# ---------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# 4. 학습 + 검증
# ---------------------------
num_epochs = 2  # 테스트용, 속도 위해 2로 설정
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")

    # 검증
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")

Epoch 1/2, Train Loss: 2.0075
Validation Accuracy: 1.0000
Epoch 2/2, Train Loss: 0.3378
Validation Accuracy: 1.0000


In [ ]:
# 5. 모델 저장
# ---------------------------
torch.save(model.state_dict(), "plant_model.pth")
print("Model saved as plant_model.pth")

Model saved as plant_model.pth


In [ ]:
test_dir = "/content/plant_dataset/test/test"
print(os.listdir(test_dir))  # 디렉토리 안 파일 목록 확인


['TomatoYellowCurlVirus2.JPG', 'CornCommonRust2.JPG', 'AppleCedarRust1.JPG', 'TomatoEarlyBlight5.JPG', 'TomatoEarlyBlight1.JPG', 'TomatoEarlyBlight2.JPG', 'TomatoYellowCurlVirus3.JPG', 'PotatoEarlyBlight4.JPG', 'TomatoYellowCurlVirus1.JPG', 'TomatoYellowCurlVirus5.JPG', 'AppleScab2.JPG', 'PotatoEarlyBlight3.JPG', 'TomatoYellowCurlVirus4.JPG', 'CornCommonRust3.JPG', 'AppleCedarRust2.JPG', 'PotatoHealthy2.JPG', 'TomatoEarlyBlight6.JPG', 'PotatoHealthy1.JPG', 'AppleScab3.JPG', 'AppleCedarRust3.JPG', 'TomatoEarlyBlight4.JPG', 'PotatoEarlyBlight1.JPG', 'TomatoEarlyBlight3.JPG', 'TomatoHealthy3.JPG', 'TomatoHealthy4.JPG', 'AppleCedarRust4.JPG', 'TomatoYellowCurlVirus6.JPG', 'AppleScab1.JPG', 'TomatoHealthy1.JPG', 'TomatoHealthy2.JPG', 'CornCommonRust1.JPG', 'PotatoEarlyBlight2.JPG', 'PotatoEarlyBlight5.JPG']


In [ ]:
# 저장된 모델 불러오기
model.load_state_dict(torch.load("plant_model.pth", map_location=device))
model.eval()
print("Model loaded successfully")

Model loaded successfully


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# 전처리 (train/valid과 동일해야 함)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# test_dir 안의 이미지 하나 선택
test_dir = "/content/plant_dataset/test/test"
test_image_path = os.path.join(test_dir, "TomatoYellowCurlVirus2.JPG")

# 이미지 열기
img = Image.open(test_image_path)
img = transform(img).unsqueeze(0).to(device)

# 예측
with torch.no_grad():
    outputs = model(img)
    _, predicted = outputs.max(1)

print("예측한 클래스 인덱스:", predicted.item())

예측한 클래스 인덱스: 0


In [ ]:
#문제
from torch.utils.data import DataLoader

# test 데이터셋 로드
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

test_dataset = datasets.ImageFolder("/content/plant_dataset/test", transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 평가
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 100.00%


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 테스트 데이터셋 및 전처리
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_dir = "/content/plant_dataset/test/test"  # 여기서 클래스 폴더들이 있음
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# 3. 모델 불러오기
model.load_state_dict(torch.load("plant_model.pth"))
model.to(device)
model.eval()

# 4. 테스트 정확도 계산
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# 5. 이미지별 예측 결과 확인
class_names = test_dataset.classes
for i in range(len(test_dataset)):
    image, label = test_dataset[i]
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
    print(f"{test_dataset.imgs[i][0].split('/')[-1]} → Predicted class: {class_names[pred.item()]}")

    #==============================================================================================================================================까지 이전


Using device: cuda
Test Accuracy: 12.12%
AppleCedarRust1.JPG → Predicted class: AppleCedarRust
AppleCedarRust2.JPG → Predicted class: AppleCedarRust
AppleCedarRust3.JPG → Predicted class: AppleCedarRust
AppleCedarRust4.JPG → Predicted class: AppleCedarRust
AppleScab1.JPG → Predicted class: AppleCedarRust
AppleScab2.JPG → Predicted class: AppleCedarRust
AppleScab3.JPG → Predicted class: AppleCedarRust
CornCommonRust1.JPG → Predicted class: AppleCedarRust
CornCommonRust2.JPG → Predicted class: AppleCedarRust
CornCommonRust3.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight1.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight2.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight3.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight4.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight5.JPG → Predicted class: AppleCedarRust
PotatoHealthy1.JPG → Predicted class: AppleCedarRust
PotatoHealthy2.JPG → Predicted class: AppleCedarRust
TomatoEarlyBlight1.JPG → Predicted class: A

In [ ]:
!rm -rf /content/plant_dataset/test/test/.ipynb_checkpoints

In [ ]:
test_dir = "/content/plant_dataset/test/test"
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

In [ ]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# 클래스별 예측 출력
for img_path, (img, _) in zip(test_dataset.imgs, test_dataset):
    image = img.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
    print(f"{img_path[0].split('/')[-1]} → Predicted class: {test_dataset.classes[pred.item()]}")


Test Accuracy: 12.12%
AppleCedarRust1.JPG → Predicted class: AppleCedarRust
AppleCedarRust2.JPG → Predicted class: AppleCedarRust
AppleCedarRust3.JPG → Predicted class: AppleCedarRust
AppleCedarRust4.JPG → Predicted class: AppleCedarRust
AppleScab1.JPG → Predicted class: AppleCedarRust
AppleScab2.JPG → Predicted class: AppleCedarRust
AppleScab3.JPG → Predicted class: AppleCedarRust
CornCommonRust1.JPG → Predicted class: AppleCedarRust
CornCommonRust2.JPG → Predicted class: AppleCedarRust
CornCommonRust3.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight1.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight2.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight3.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight4.JPG → Predicted class: AppleCedarRust
PotatoEarlyBlight5.JPG → Predicted class: AppleCedarRust
PotatoHealthy1.JPG → Predicted class: AppleCedarRust
PotatoHealthy2.JPG → Predicted class: AppleCedarRust
TomatoEarlyBlight1.JPG → Predicted class: AppleCedarRust
Tomat

In [ ]:
# 1. 라이브러리 임포트
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 테스트 이미지 전처리
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 3. 테스트 데이터셋
test_dir = "/content/plant_dataset/test/test"
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# 4. 모델 불러오기 (여기서 num_classes 맞추기)
num_classes = 38  # 학습할 때 클래스 수
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("plant_model.pth", map_location=device))
model = model.to(device)
model.eval()

# 5. 테스트 정확도 계산
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total * 100:.2f}%")



Test Accuracy: 12.12%


In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# 데이터 증강: 학습 데이터만 적용
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = valid_transform

# 데이터셋 불러오기
train_dataset = datasets.ImageFolder('/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train', transform=train_transform)
valid_dataset = datasets.ImageFolder('/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid', transform=valid_transform)
test_dataset  = datasets.ImageFolder("/content/plant_dataset/test/test",  transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # 학습 중 lr 감소


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ---------------------------
# 1. 데이터 전처리
# ---------------------------
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder("/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train", transform=train_transform)
valid_dataset = datasets.ImageFolder("/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid", transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=2)

# ---------------------------
# 2. 모델 준비 (Feature Extraction)
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# 모든 파라미터 동결
for param in model.parameters():
    param.requires_grad = False

# 출력 레이어 수정 (클래스 수에 맞게)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)



Using device: cuda


In [ ]:
# ---------------------------
# 3. 손실함수, 옵티마이저
# ---------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)  # fc만 학습




In [ ]:
# --------------------------- ==========================여기서 문제 발생 --- 다시
# 4. 학습 + 검증 (조기 종료)
# ---------------------------
num_epochs = 10
best_valid_acc = 0
patience = 3
trigger_times = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    # 검증
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    valid_acc = correct / total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Acc: {valid_acc:.4f}")

    # 조기 종료
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), "best_plant_model.pth")  # 최고 모델 저장
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

print("Training complete. Best validation accuracy:", best_valid_acc)

KeyboardInterrupt: 